In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import math
import os

from tqdm import tqdm
from sklearn.metrics import f1_score

seed = 1
result_folder = 'results'
score_file = os.path.join(result_folder, 'scores.txt')
test_file = os.path.join(result_folder, 'test.tsv')
os.makedirs(result_folder, exist_ok=True)

# Setting seed

In [2]:
def set_seed(seed):
    torch.use_deterministic_algorithms(True)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(1)

# Data loading

In [3]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [4]:
def read_dataset(filename, splitter="\t"):
    data = []
    sentence = []
    tags = []
    with open(filename) as f:
        for line in f:
            if not line.isspace():
                word, tag = line.split(splitter)
                sentence.append(word)
                tags.append(tag.strip())
            else:
                data.append((sentence, tags))
                sentence = []
                tags = []
    return data

In [5]:
training_data = read_dataset("data/train.tsv")

In [6]:
test_data = read_dataset("data/test.tsv")

In [7]:
word_to_ix = {}


# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
            
for sent, tags in test_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index


tag_to_ix = {
    "O": 0,
    "B-Object": 1,
    "I-Object": 2,
    "B-Aspect": 3,
    "I-Aspect": 4,
    "B-Predicate": 5,
    "I-Predicate": 6
}  # Assign each tag with a unique index

idx_to_tag = dict(map(reversed, tag_to_ix.items()))

EMBEDDING_DIM = 128
HIDDEN_DIM = 256

# Tagger

In [8]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), 0, :]
        return self.dropout(x)

class Tagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super().__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.pos_encoder = PositionalEncoding(embedding_dim, dropout=0)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=4, dim_feedforward=2 * embedding_dim, dropout=0)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=3)
        #self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Sequential(nn.Linear(embedding_dim, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, tagset_size))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        embeds = self.pos_encoder(embeds)
        #lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        out = self.transformer(embeds.view(len(sentence), 1, -1))
        #out = self.lstm(out)[0]
        tag_space = self.hidden2tag(out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

# Training

In [9]:
model = Tagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.AdamW(model.parameters())

In [10]:
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

for epoch in tqdm(range(20)):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

100%|█████████████████████████████████| 20/20 [07:13<00:00, 21.65s/it]


# Inference

In [11]:
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    tags = [idx_to_tag[int(i)] for i in tag_scores.argmax(dim=-1)]
    
    for i, y in zip(training_data[0][0], tags):
        print(i, y)

also O
, O
i O
have O
recently O
discovered O
advil B-Object
liquigels O
work O
much O
better B-Predicate
and O
faster B-Predicate
for O
a O
headache B-Aspect
than O
regular O
ibuprofen B-Object
. O


In [12]:
with open(test_file, "w") as w:
    with torch.no_grad():
        for sentence in tqdm(test_data):
            inputs = prepare_sequence(sentence[0], word_to_ix)
            tag_scores = model(inputs)
            tags = [idx_to_tag[int(i)] for i in tag_scores.argmax(dim=-1)]
            for i, y in zip(sentence[0], tags):
                w.write(f"{i}\t{y}\n")
            w.write("\n")

100%|█████████████████████████████| 283/283 [00:00<00:00, 1112.40it/s]


In [13]:
from evaluation.evaluate_f1_partial import main

main('data/test.tsv', test_file, score_file)
    
with open(score_file, "r") as f:
    print(f.read())

f1_average_strict: 0.437729
f1_aspect_strict: 0.128834
f1_object_strict: 0.354149
f1_predicate_strict: 0.812500
f1_average: 0.452591
f1_aspect: 0.158998
f1_object: 0.360194
f1_predicate: 0.835016

